Importamos las librerias Necesarias

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Creamos una clase para almacenar todos los datos de la red neuronal. Utilizaremos la lógica de temperatura para controlar la creatividad del modelo.
Esta logica se basa en la formula: $$
T(x) = \frac{\exp(x / T)}{\sum \exp(x_i / T)}
$$
 * La temperatura ajusta la distribución de probabilidad de las predicciones del modelo. 
 * Cuando la temperatura es alta (>1), las probabilidades se distribuyen más uniformemente, 
   lo que da como resultado respuestas más diversas.
 * Cuando la temperatura es baja (<1), el modelo se vuelve más confiado y genera respuestas 
   más determinísticas.


In [6]:
class TextGeneratorRNN:
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        
        # Construcción del modelo con Keras
        self.model = Sequential([
            Embedding(input_dim=vocab_size, output_dim=embedding_dim),
            SimpleRNN(hidden_size, activation='tanh', return_sequences=True),
            Dense(vocab_size, activation='softmax')
        ])
        
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    def train(self, X, y, epochs=10, batch_size=32):
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size)
    
    def generate_text(self, seed_text, tokenizer, max_length=100, temperature=1.0):
        generated_text = seed_text
        for _ in range(max_length):
            tokenized_input = tokenizer.texts_to_sequences([generated_text])
            tokenized_input = pad_sequences(tokenized_input, maxlen=max_length, padding='pre')
            
            predictions = self.model.predict(tokenized_input, verbose=0)[0][-1]
            predictions = np.log(predictions + 1e-8) / temperature
            exp_preds = np.exp(predictions)
            probabilities = exp_preds / np.sum(exp_preds)
            
            next_word_index = np.random.choice(len(probabilities), p=probabilities)
            next_word = tokenizer.index_word.get(next_word_index, '')
            
            if not next_word:
                break
            
            generated_text += ' ' + next_word
        
        return generated_text


Para esta red vamos a utilizar el dataset de Shakespeare que contiene dialogos y textos de varias obras de Shakespeare

In [7]:
# Cargar y preprocesar el dataset de Shakespeare
def load_shakespeare_data(seq_length=50):
    path = tf.keras.utils.get_file("shakespeare.txt", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read().lower()
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text])
    sequences = tokenizer.texts_to_sequences([text])[0]
    vocab_size = len(tokenizer.word_index) + 1
    
    input_sequences = []
    for i in range(len(sequences) - seq_length):
        input_sequences.append(sequences[i:i+seq_length+1])  # Ventana deslizante fija

    input_sequences = np.array(input_sequences)
    X, y = input_sequences[:, :-1], input_sequences[:, 1:]  # Desplazamos las etiquetas
    
    return X, y, vocab_size, tokenizer


In [8]:
# Cargar datos
X, y, vocab_size, tokenizer = load_shakespeare_data()

In [9]:
# Crear y entrenar el modelo
model = TextGeneratorRNN(vocab_size, embedding_dim=64, hidden_size=128)
model.train(X, y, epochs=10, batch_size=32)

Epoch 1/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 798s 125ms/step - accuracy: 0.0849 - loss: 6.0464
Epoch 2/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 789s 124ms/step - accuracy: 0.2771 - loss: 3.8413
Epoch 3/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 790s 124ms/step - accuracy: 0.3841 - loss: 3.1336
Epoch 4/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 790s 124ms/step - accuracy: 0.4336 - loss: 2.8336
Epoch 5/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 793s 124ms/step - accuracy: 0.4626 - loss: 2.6679
Epoch 6/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 794s 124ms/step - accuracy: 0.4805 - loss: 2.5669
Epoch 7/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 794s 124ms/step - accuracy: 0.4939 - loss: 2.4920
Epoch 8/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 794s 125ms/step - accuracy: 0.5042 - loss: 2.4363
Epoch 9/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 795s 125ms/step - accuracy: 0.5115 - loss: 2.3946
Epoch 10/10
6377/6377 ━━━━━━━━━━━━━━━━━━━━ 814s 128ms/step - accuracy: 0.5178 - loss: 2.3612


In [10]:
# Generar texto de ejemplo
seed_text = "to be or not to be"
generated_text = model.generate_text(seed_text, tokenizer, max_length=50, temperature=0.7)
print("Generated Text:\n", generated_text)


Generated Text:
 to be or not to be his country page and that i were a mockery king of snow standing before the sun of love and others live sister hither widow miserable day clown then sit richard presently if you be gone away with you sir ay good sir good letters kate and be it like a
